## **CNN parser:** ##

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [68]:
EMAIL = "username@email.com"
PASSWORD = "my_password"

In [77]:
driver = webdriver.Firefox()
driver.get("https://edition.cnn.com/account/log-in")
wait = WebDriverWait(driver, 10)

# cookie banner closing:
try:
    cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Accept All')]")))
    cookie_button.click()
    print('Banner was passed...\n')
    time.sleep(2)
except Exception:
    print('Cookie banner didn\'t appear...\n')

# logging:
email_input = wait.until(EC.element_to_be_clickable((By.NAME, "loginEmail")))
email_input.send_keys(EMAIL)

password_input = wait.until(EC.element_to_be_clickable((By.NAME, "loginPassword")))
password_input.send_keys(PASSWORD)

# 'Sign In' button clicking:
login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Sign in')]")))
login_button.click()
time.sleep(5)

# moving to the current url:
driver.get("https://edition.cnn.com/business")
time.sleep(2)

news_hrefs = list(set([a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, "a[data-link-type='article']")]))
news_hrefs = news_hrefs[:10]

news_headliners = []
publishing_dates = []
news_authors = []
sources = []
for news_href in news_hrefs:
    driver.get(news_href)
    time.sleep(2)
    
    news_headliner = driver.find_element(By.CSS_SELECTOR, "h1[data-editable='headlineText']").text
    publish_date = driver.find_element(By.CSS_SELECTOR, "div[data-uri*='_components/timestamp/']").text
    news_author = driver.find_element(By.CSS_SELECTOR, "div[class^='byline__name']").text

    author_parts = news_author.rsplit(" ", 1)
    news_author = author_parts[0]
    source = author_parts[1] if len(author_parts) > 1 else 'CNN' # to get the current sources from 'news_author'
    
    publishing_dates.append(publish_date)
    news_headliners.append(news_headliner)
    news_authors.append(news_author)
    sources.append(source)
    
    print(f'{news_headliner} -> {publish_date} -> {news_author} -> {source}\n')
    
print('done')
driver.quit()

Banner was passed...

China’s foreign investment sank in January. Can Beijing turn things around? -> Published 2:21 AM EST, Thu February 20, 2025 -> By John Liu, -> CNN

Washington Post opted not to publish anti-Musk ad campaign after initially being open -> Published 2:02 PM EST, Mon February 17, 2025 -> By Liam Reilly, -> CNN

HSBC to save $1.8 billion over two years under new CEO -> Published 1:28 AM EST, Wed February 19, 2025 -> By -> Reuters

The Trump administration is reversing Biden policies everywhere. But it’s keeping this one -> Published 5:00 AM EST, Thu February 20, 2025 -> By Nathaniel Meyersohn, -> CNN

Elon Musk debuts Grok 3, an AI model that he says outperforms ChatGPT and DeepSeek -> Updated 1:39 PM EST, Tue February 18, 2025 -> By Clare Duffy, -> CNN

Elon Musk will withdraw his nearly $100 billion bid for OpenAI if it remains a nonprofit -> Updated 7:05 AM EST, Thu February 13, 2025 -> By David Goldman, -> CNN

In from the cold? Alibaba co-founder Jack Ma spotted a

In [83]:
print(news_authors)

clean_authors = [] # need to process the sources...
for author in news_authors:
    author = author.replace("By ", "").strip(",").strip()
    if author.lower() == "by" or not author:
        author = "Author wasn\'t found"
    clean_authors.append(author)

print()
print(clean_authors)

['By John Liu,', 'By Liam Reilly,', 'By', 'By Nathaniel Meyersohn,', 'By Clare Duffy,', 'By David Goldman,', 'By John Liu,', 'By Clare Duffy and Diksha Madhok,', 'By Ramishah Maruf,', 'By Jordan Valinsky,']

['John Liu', 'Liam Reilly', "Author wasn't found", 'Nathaniel Meyersohn', 'Clare Duffy', 'David Goldman', 'John Liu', 'Clare Duffy and Diksha Madhok', 'Ramishah Maruf', 'Jordan Valinsky']


In [84]:
df = pd.DataFrame({
    "Headline": news_headliners,
    "Date": publishing_dates,
    "Author": clean_authors,
    "Source": sources
})

df.to_csv("cnn_news.csv", index = False, encoding = "utf-8-sig")
print('all data was saved!\n')

all data was saved!



In [85]:
# ...